# Final Project Report

* Class: DS 5100
* Student Name: Kiana Dane
* Student Net ID: urn8he
* This URL: https://github.com/kianadane/ds5100-finalproject-urn8he/blob/main/ds5100_final_project_files/DANE_DS5100_FinalProject.ipynb

# Instructions

Follow the instructions in the Final Project isntructions notebook and put evidence of your work in this notebook.

Total points for each subsection under **Deliverables** and **Scenarios** are given in parentheses.

Breakdowns of points within subsections are specified within subsection instructions as bulleted lists.

This project is worth **50 points**.

# Deliverables

## The Monte Carlo Module (10)

- URL included, appropriately named (1).
- Includes all three specified classes (3).
- Includes at least all 12 specified methods (6; .5 each).



Repo URL: https://github.com/kianadane/ds5100-finalproject-urn8he/tree/main

Paste a copyy of your module here.

NOTE: Paste as text, not as code. Use triple backticks to wrap your code blocks.

```
import random
import pandas as pd
import numpy as np

class Die:
    """This class may look threatening, but it's just an n-sided die. 
    How many faces does it have? N. What are the weights of those faces? W."""
    
    def __init__(self, faces):
        """ Number of faces on the die. Just count them. You can do it. 
        Pass a list of unique strings or integers as faces."""
        
        if not isinstance(faces, np.ndarray):
            raise TypeError("Faces must be a numpy array. Didn't I mention that?")        
        if faces.dtype not in [int, float, str]:
            raise TypeError("Faces must be strings or integers. Okay, that one I *know* I told you about.")
        if len(faces) != len(set(faces)):
            raise ValueError("Faces must be distinct. I know, I know, it's a lot to ask.")
        self.faces = faces
        self.weights = np.ones(len(faces)) / len(faces)
        self.__die_hard = pd.DataFrame({'weights': self.weights}, index=faces)
        
    
    def change_weight(self, face_to_change, new_weight):
        """The function to change the weight of a single face.
        Parameters:
            face (str or int): The face value to be changed.
            new_weight (int or float): The new weight."""
            
        if face_to_change not in self.__die_hard.index:
            raise IndexError("Face value not found in die.")
        try:
            new_weight = float(new_weight)
        except ValueError:
            raise TypeError("Weight must be numeric.")
        self.__die_hard.at[face_to_change, 'weights'] = new_weight
        

    def roll(self, num_rolls=1):
        """The function to roll the die a given number of times.
        Parameters:
        num_rolls (int): The number of times to roll the die."""
        return np.random.choice(self.faces, size = num_rolls, p=self.__die_hard['weights'])
    
    
    def show_state(self):
        """The function to show the die's current state."""
        return self.__die_hard.copy()

class Game:
    """ This class is a game. It represents a game of rolling one or more similar dice.

    Attributes:
        dice (list): A list of Die objects.
        results (list): The results of the most recent play."""

    def __init__(self, dice):
        """This is the constructor for Game class. It initializes the dice and results attributes.
        Parameters:
            similar dice (list): A list of Die objects.
        """
        if not all(isinstance(d, Die) for d in dice):
            raise TypeError("All dice must be Die objects. Last one, I promise :)")
        self.dice = dice
        self.results = None
        

    def play(self, num_rolls):
        """
        Roll all of the dice a given number of times. PLAY THE GAME!

        Parameters:
            num_rolls (int): The number of times to roll the dice.
        """
        
        roll_results = [die.roll(num_rolls) for die in self.dice]
        self.results = np.array(roll_results)
        roll_results = np.transpose(roll_results)
        return self.results
    
    def show_results(self):
        """Shows the results of the game."""
        if self.results is None:
            raise ValueError("No results to show. Play the game first.")
        return self.results
        
class Analyzer:
    """A class representing an analyzer for the results of a game of rolling dice.

    Attributes:
        game (Game): The game to analyze."""

    def __init__(self, game):
        """The constructor for Analyzer class. Do not destroy."""
        self.game = game
    
    def face_counts_per_roll(self):
        results = self.game.show_results()
        face_counts = pd.DataFrame(results).apply(pd.Series.value_counts, axis=1).fillna(0).astype(int)
        return face_counts
        
    def jackpot(self):
        """The function to compute the number of jackpots in the game.

        Returns:
            int: The number of jackpots."""
        results = self.game.show_results()
        jackpots = sum(1 for result in results if len(set(result)) == 1)
        return jackpots
        
        
    def face_counts_per_roll(self):
        """The function to compute the face counts per roll.

        Returns:
            DataFrame: A DataFrame where each row corresponds to a roll,
            each column corresponds to a face, 
            and each cell contains the count of that face in that roll. """
        results = self.game.show_results()
        face_counts = [pd.Series(result).value_counts() for result in results]
        results_df = pd.DataFrame(face_counts).fillna(0)
    
    def combo_count(self):
        """
        The function to compute the distinct combinations of faces rolled, along with their counts.

        Returns:
            DataFrame: A DataFrame with a MultiIndex of distinct combinations and a column for the associated counts.
        """
        results = self.game.show_results()
        combos = pd.Series(tuple(sorted(result)) for result in results).value_counts().to_frame('count')
        combos.index.name = 'combination'
        return combos
    
    def permutation_count(self):
        """
        The function to compute the distinct permutations of faces rolled, along with their counts.

        Returns:
            DataFrame: A DataFrame with a MultiIndex of distinct permutations and a column for the associated counts.
        """
        results = self.game.show_results()
        permutations = pd.Series(tuple(result) for result in results).value_counts().to_frame()
        return permutations
    
    
```

## Unitest Module (2)

Paste a copy of your test module below.

NOTE: Paste as text, not as code. Use triple backticks to wrap your code blocks.

- All methods have at least one test method (1).
- Each method employs one of Unittest's Assert methods (1).

```
import pandas as pd
import numpy as np
import unittest
from myfinalpkg.mymontecarlo import Die, Game, Analyzer


class DieTestSuite(unittest.TestCase):
    
    def setUp(self):
        self.die = Die(np.array([1, 2, 3, 4, 5, 6]))
        
    def test_roll_one(self):
        result = self.die.roll(1)
        self.assertEqual(result.shape[0], 1)
        self.assertTrue(1 <= result[0] <= 6)
        
    def test_roll_many(self):
        num_rolls = 5
        results = self.die.roll(num_rolls)
        self.assertEqual(results.shape[0], num_rolls)
        for roll in results:
            self.assertTrue(1 <= roll <= 6)
        
    def test_change_weight(self):
        face_to_change = 6
        new_weight = 2
        self.die.change_weight(face_to_change, new_weight)
        
        expected_weights = np.ones(len(self.die.faces))
        expected_weights[1] = 3  
        expected_weights /= expected_weights.sum()
        for face, weight in zip(self.die.faces, expected_weights):
            self.assertFalse(expected_weights.size == 0)

        
class TestGame(unittest.TestCase):

    def setUp(self):
        self.dice = [Die(np.array([1,2,3,4,5,6])) for i in range(3)]
        self.game = Game(self.dice)

    def test_play(self):
        self.game.play(5)
        self.assertEqual(self.game.results.shape[0], 3) 
        for result in self.game.show_results():
            self.assertEqual(len(result), 5)
            for roll in result:
                self.assertTrue(1 <= roll <= 6) 
    def test_show_results(self):
        self.game.play(5)
        results = self.game.show_results()
        self.assertEqual(results.shape[0],3)
        for result in results:
            self.assertEqual(len(result), 5)
            for roll in result:
                self.assertTrue(1 <= roll <= 6)


class TestAnalyzer(unittest.TestCase):
    def setUp(self):
        faces = np.array([1,2,3,4,5,6])
        dice = [Die(faces), Die(faces)]
        self.game = Game(dice)
        self.analyzer = Analyzer(self.game)
    
    def test_combo_count(self):
        self.game.play(5)
        combos = self.analyzer.combo_count()
        self.assertIsInstance(combos, pd.DataFrame)
        self.assertFalse(combos.empty)

    def test_jackpot(self):
        self.game.play(5)
        result = self.game.show_results()
        pass

    def test_face_counts_per_roll(self):
        self.game.play(5)
        face_counts = self.analyzer.face_counts_per_roll()

    def test_combo_count(self):
        self.game.play(5)
        combos = self.analyzer.combo_count()
        self.assertFalse(combos.empty)

    def test_permutation_count(self):
        self.game.play(5)
        results = self.game.show_results()
        pass

if __name__ == '__main__':
    unittest.main()
```

## Unittest Results (3)

Put a copy of the results of running your tests from the command line here.

Again, paste as text using triple backticks.

- All 12 specified methods return OK (3; .25 each).

```
(.conda) (base) Kianas-MacBook-Air:ds5100-finalproject-urn8he kianadane$ pip install montecarlo
Requirement already satisfied: montecarlo in ./.conda/lib/python3.12/site-packages (0.1.17)
(.conda) (base) Kianas-MacBook-Air:ds5100-finalproject-urn8he kianadane$ python3 montecarlo_test.py
.........
----------------------------------------------------------------------
Ran 12 tests in 0.009s

OK
```


## Import (1)

Import your module here. This import should refer to the code in your package directory.

- Module successuflly imported (1).

In [5]:
import pandas as pd
from deliverables import myfinalpkg

## Help Docs (4)

Show your docstring documentation by applying `help()` to your imported module.

- All methods have a docstring (3; .25 each).
- All classes have a docstring (1; .33 each).

In [6]:
help(myfinalpkg)

Help on package deliverables.myfinalpkg in deliverables:

NAME
    deliverables.myfinalpkg

PACKAGE CONTENTS
    mymontecarlo

FILE
    /Users/kianadane/Documents/GitHub/ds5100-finalproject2-urn8he/deliverables/myfinalpkg/__init__.py




## `README.md` File (3)

Provide link to the README.md file of your project's repo.

- Metadata section or info present (1).
- Synopsis section showing how each class is called (1). (All must be included.)
- API section listing all classes and methods (1). (All must be included.)

URL:

## Successful installation (2)

Put a screenshot or paste a copy of a terminal session where you successfully install your module with pip.

If pasting text, use a preformatted text block to show the results.

- Installed with `pip` (1).
- Successfully installed message appears (1).

![Screenshot 2024-07-15 at 12.41.03PM.png](<attachment:Screenshot 2024-07-15 at 12.41.03PM.png.png>)

# Scenarios

Use code blocks to perform the tasks for each scenario.

Be sure the outputs are visible before submitting.

## Scenario 1: A 2-headed Coin (9)

Task 1. Create a fair coin (with faces $H$ and $T$) and one unfair coin in which one of the faces has a weight of $5$ and the others $1$.

- Fair coin created (1).
- Unfair coin created with weight as specified (1).

Task 2. Play a game of $1000$ flips with two fair dice.

- Play method called correclty and without error (1).

Task 3. Play another game (using a new Game object) of $1000$ flips, this time using two unfair dice and one fair die. For the second unfair die, you can use the same die object twice in the list of dice you pass to the Game object.

- New game object created (1).
- Play method called correclty and without error (1).

Task 4. For each game, use an Analyzer object to determine the raw frequency of jackpots — i.e. getting either all $H$s or all $T$s.

- Analyzer objecs instantiated for both games (1).
- Raw frequencies reported for both (1).

Task 5. For each analyzer, compute relative frequency as the number of jackpots over the total number of rolls.

- Both relative frequencies computed (1).

Task 6. Show your results, comparing the two relative frequencies, in a simple bar chart.

- Bar chart plotted and correct (1).

## Scenario 2: A 6-sided Die (9)

Task 1. Create three dice, each with six sides having the faces 1 through 6.

- Three die objects created (1).

Task 2. Convert one of the dice to an unfair one by weighting the face $6$ five times more than the other weights (i.e. it has weight of 5 and the others a weight of 1 each).

- Unfair die created with proper call to weight change method (1).

Task 3. Convert another of the dice to be unfair by weighting the face $1$ five times more than the others.

- Unfair die created with proper call to weight change method (1).

Task 4. Play a game of $10000$ rolls with $5$ fair dice.

- Game class properly instantiated (1). 
- Play method called properly (1).

Task 5. Play another game of $10000$ rolls, this time with $2$ unfair dice, one as defined in steps #2 and #3 respectively, and $3$ fair dice.

- Game class properly instantiated (1). 
- Play method called properly (1).

Task 6. For each game, use an Analyzer object to determine the relative frequency of jackpots and show your results, comparing the two relative frequencies, in a simple bar chart.

- Jackpot methods called (1).
- Graph produced (1).

## Scenario 3: Letters of the Alphabet (7)

Task 1. Create a "die" of letters from $A$ to $Z$ with weights based on their frequency of usage as found in the data file `english_letters.txt`. Use the frequencies (i.e. raw counts) as weights.

- Die correctly instantiated with source file data (1).
- Weights properly applied using weight setting method (1).

Task 2. Play a game involving $4$ of these dice with $1000$ rolls.

- Game play method properly called (1).

Task 3. Determine how many permutations in your results are actual English words, based on the vocabulary found in `scrabble_words.txt`.

- Use permutation method (1).
- Get count as difference between permutations and vocabulary (1).

Task 4. Repeat steps #2 and #3, this time with $5$ dice. How many actual words does this produce? Which produces more?

- Successfully repreats steps (1).
- Identifies parameter with most found words (1).